In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import seaborn as sns
from itertools import product
current_dir = os.getcwd()
parallel_folder = os.path.abspath(os.path.join(current_dir, '../Fit_and_Predict'))
sys.path.append(parallel_folder)

from config import get_model_ds_kwargs,get_dataset_kwargs

/Users/leif/miniforge3/envs/hnam1/lib/python3.10/site-packages/pytorch_forecasting/models/base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
DATASET = 'Walmart'
causals = get_model_ds_kwargs('HNAM', DATASET)['causal']

In [3]:
hnam = pd.read_pickle(f'{DATASET}/hnam.pkl')
hnam['reverse_hierarchy'] = False
hnamr = pd.read_pickle(f'{DATASET}/hnam_r.pkl')
hnamr['reverse_hierarchy'] = True
scaling = pd.read_pickle(f'../Processed/{DATASET}/{DATASET.lower()}_scaling.pkl')

data = pd.concat([hnam, hnamr], axis=0)

data['mean'] = data['time_series'].map(scaling['mean'])
data['std'] = data['time_series'].map(scaling['std'])
data['pred_scaled'] = (data['pred']  - data['mean'])  /  data['std']
data['true_scaled'] = (data['true']  - data['mean']) /  data['std']
data['srmse'] = (data['true_scaled'] - data['pred_scaled']) ** 2
data.groupby(['reverse_hierarchy'])['srmse'].mean()**0.5

reverse_hierarchy
False    0.846452
True     0.848384
Name: srmse, dtype: float32

In [4]:
hnam = pd.read_pickle(f'{DATASET}/hnam_all.pkl')
hnam['reverse_hierarchy'] = False
hnamr = pd.read_pickle(f'{DATASET}/hnamR_all.pkl')
hnamr['reverse_hierarchy'] = True
scaling = pd.read_pickle(f'../Processed/{DATASET}/{DATASET}_scaling.pkl')

data = pd.concat([hnam, hnamr], axis=0)

effect_cols = hnam.columns[hnam.columns.str.contains('effect')]
scaled_cols = [f'{col}_scaled' for col in effect_cols]
data['mean'] = data['time_series'].map(scaling['mean'])
data['std'] = data['time_series'].map(scaling['std'])
for col in effect_cols:
    data[f'{col}_scaled'] = (data[col]  - data['mean'])  /  data['std']

In [5]:
diff = data[data['reverse_hierarchy']==True][scaled_cols].reset_index(drop=True) - data[data['reverse_hierarchy']==False][scaled_cols].reset_index(drop=True)

In [6]:
for i in range(len(effect_cols)):
    data.loc[data[effect_cols[i]]==0,[scaled_cols[i]]] = 0

In [7]:
data[data['reverse_hierarchy']==True][scaled_cols].std().round(3)

effect_level_pred_scaled    0.470
effect_weekday_scaled       0.578
effect_snap_scaled          0.886
effect_National_scaled      0.733
effect_Cultural_scaled      0.622
effect_Sporting_scaled      0.423
effect_relprice_scaled      0.376
dtype: float32

In [8]:
((diff.replace(0,np.nan)**2).mean()**0.5).round(3)

effect_level_pred_scaled    0.079
effect_weekday_scaled       0.096
effect_snap_scaled          0.122
effect_National_scaled      0.190
effect_Cultural_scaled      0.189
effect_Sporting_scaled      0.273
effect_relprice_scaled      0.128
dtype: float32